In [1]:
import sys
sys.path.append('/home/marnugue/Documents/crop_scheduling_optimization')
from src.models.py_crop_engine import PYCropEngine
engine = PYCropEngine()

import time
start_time = time.time()
engine.execute(time_limit=30,
               solution_path="./",
               solver='scip')
print("--- Execution takes %s seconds ---" % (time.time() - start_time))

Scheduling ...
Scheduler find feasible solution!
Solution is optimal!


--- Execution takes 20.10602641105652 seconds ---


In [14]:
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

pd.options.mode.copy_on_write = True

START_DATE = datetime(2023,10,31)

In [15]:
df = pd.read_csv("./solution.csv", header=[0,1], index_col=0)

df_list = []
df = df.iloc[1:,:]
for col in df.columns:
    if (df[col] == 0).all():
        df.drop(col,axis=1,inplace=True)
        
def get_vegetable(row):
    l = [idx for idx in row.index if row[idx]]
    if l:
        return l[0]
    else:
        return None

for pos in df.columns.get_level_values(level=0):
    df_pos = df[pos]
    df_pos['Vegetable'] = df_pos.apply(lambda row: get_vegetable(row), axis=1)
    df_pos['Event'] = (df_pos != df_pos.shift(1)).T.any()
    df_pos['Event'] = df_pos['Event'].cumsum()
    df_start = df_pos.reset_index()[['Time','Event']].groupby('Event').first()
    df_end = df_pos.reset_index()[['Time','Event']].groupby('Event').last()
    df_start_end = pd.merge(df_start,
                        df_end,
                        left_index=True,
                        right_index=True,
                        how='inner',
                        suffixes=('_start','_end'))
    df_start_end['Time_end'] = df_start_end['Time_end'] + 1
    df_start_end = df_start_end.apply(lambda row: START_DATE + pd.to_timedelta(row,'d'))
    df_pos = pd.merge(df_pos,
                    df_start_end,
                    left_on='Event',
                    right_index=True,
                    how='inner')
    df_pos.drop_duplicates(inplace=True)
    df_pos['Pos'] = pos
    df_list.append(df_pos[['Pos','Vegetable','Time_start','Time_end']])
df = pd.concat(df_list,ignore_index=True)

In [16]:
df

Vegetable,Pos,Vegetable,Time_start,Time_end
0,P0,H3,2023-11-01,2023-11-04
1,P0,H1,2023-11-04,2023-11-09
2,P0,H2,2023-11-09,2023-11-14
3,P0,H1,2023-11-14,2023-11-19
4,P0,H3,2023-11-19,2023-11-21
...,...,...,...,...
73,P3,H3,2023-11-01,2023-11-05
74,P3,H2,2023-11-05,2023-11-09
75,P3,H1,2023-11-09,2023-11-20
76,P3,H2,2023-11-20,2023-11-27


In [19]:
fig = px.timeline(df, x_start='Time_start',
                  x_end='Time_end',
                  y='Pos',
                  color='Vegetable')
fig.add_vline(x="2023-11-15")
fig.add_vline(x="2023-11-16")
fig.add_vline(x="2023-11-29")
fig.show()